#Импорт библиотек

In [14]:
import pandas as pd  #для работы с табличными данными
import joblib  #для загрузки сериализованной модели машинного обучения

In [15]:
test_df = pd.read_excel("data/test.xlsx")  #
model = joblib.load("LGBMRegressor.pkl")  #загружается заранее обученная модель
feature_columns = model.feature_names_in_  #метод feature_names_in_ возвращает список признаков, которые использовались при обучении модели
#(для того, чтобы данные в тесте соответствовали тем же признакам)

In [16]:
print("Ожидаемые признаки:", feature_columns)  #выводим ожидаемые моделью признаки и реальные колонки test_df для проверки их соответствия
print("Колонки test_df:", test_df.columns)

Ожидаемые признаки: ['year' 'month' 'day' 'week' 'Price_source' 'Price_Diff']
Колонки test_df: Index(['dt', 'Цена на арматуру'], dtype='object')


#Преобразование временных данных

In [17]:
if "dt" in test_df.columns:
    test_df["dt"] = pd.to_datetime(test_df["dt"])
    test_df["year"] = test_df["dt"].dt.year
    test_df["month"] = test_df["dt"].dt.month
    test_df["day"] = test_df["dt"].dt.day
    test_df["week"] = test_df["dt"].dt.isocalendar().week

#Новые признаки на основе цены

In [18]:
if "Цена на арматуру" in test_df.columns:
    test_df["Price_source"] = test_df["Цена на арматуру"].shift(1)  #цена на предыдущей неделе
    test_df["Price_Diff"] = test_df["Цена на арматуру"].diff()  #разница цены по сравнению с предыдущей неделей

    #заполняем NaN средними значениями соседних строк
    test_df["Price_source"] = test_df["Price_source"].interpolate(method="linear")
    test_df["Price_Diff"] = test_df["Price_Diff"].interpolate(method="linear")

    #если остались NaN, заменяем их средним значением
    test_df["Price_source"] = test_df["Price_source"].fillna(test_df["Price_source"].mean())
    test_df["Price_Diff"] = test_df["Price_Diff"].fillna(test_df["Price_Diff"].mean())

#Дополнительный вывод информации

In [19]:
feature_columns = model.feature_names_in_
print("Ожидаемые признаки:", feature_columns)
print("Колонки test_df:", test_df.columns)

Ожидаемые признаки: ['year' 'month' 'day' 'week' 'Price_source' 'Price_Diff']
Колонки test_df: Index(['dt', 'Цена на арматуру', 'year', 'month', 'day', 'week',
       'Price_source', 'Price_Diff'],
      dtype='object')


#Подготовка данных для предсказания

In [20]:
X_test = test_df[feature_columns]  #создаём новый датафрейм X_test, содержащий только те признаки, которые использовались при обучении модели 
test_df["Прогноз_цены"] = model.predict(X_test)

#Функция для расчёта количества недель закупки

In [21]:
def calculate_weeks(row):
    if row["Прогноз_цены"] > row["Цена на арматуру"] * 1.02:  #если прогнозируемая цена вырастет более чем на 2% закупка на 6 недель (запас, чтобы избежать дорогих покупок в будущем)
        return 6
    elif row["Прогноз_цены"] > row["Цена на арматуру"]:  #если цена слегка повысится (но не выше 2%) закупка на 4 недели
        return 4
    elif row["Прогноз_цены"] < row["Цена на арматуру"] * 0.98:  #если цена снизится более чем на 2% → закупаем только на 1 неделю, чтобы позже купить дешевле.
        return 1
    else:
        return 3  #если цена остаётся стабильной → закупаем на 3 недели.
    
test_df["Закупка_на_колво_недель"] = test_df.apply(calculate_weeks, axis=1)  #применяеv функцию calculate_weeks к каждой строке датафрейма

In [22]:
X_test = test_df[feature_columns]

In [23]:
test_df["Прогноз_цены"] = model.predict(X_test).round().astype(int)  #предсказания цены на арматуру на следующую неделю, округление и приведение предсказания к целому числу
test_df = test_df.drop(columns=["Прогноз_недель"], errors="ignore")  #удаление прошлых косяков


In [24]:
test_df.head()  #выводим первые 5 строк

,dt,Цена на арматуру,year,month,day,week,Price_source,Price_Diff,Прогноз_цены,Закупка_на_колво_недель
0,2022-09-05,46000,2022,9,5,36,43833.333333,351.851852,42795,1
1,2022-09-12,46000,2022,9,12,37,46000.000000,0.000000,46060,4
2,2022-09-19,46000,2022,9,19,38,46000.000000,0.000000,46132,4
3,2022-09-26,45500,2022,9,26,39,46000.000000,-500.000000,46645,6
4,2022-10-03,45500,2022,10,3,40,45500.000000,0.000000,44907,3


In [25]:
test_df.to_excel("predicted_procurement.xlsx", index=False)  #сохраняем результат в формате xlsx

print("Прогнозирование завершено. Результаты сохранены в predicted_procurement.xlsx")

Прогнозирование завершено. Результаты сохранены в predicted_procurement.xlsx
